### 데이터
: 일정 시간 간격으로 생성되는 여러 개의 가격봉에 대한 지지선과 저항선 구하기

#### 지지선  
: 일정 시간 간격으로 생성되는 봉의 저점들의 평균  

<br/>  

> 지지선 구하기  

In [1]:
import requests
import json
import yaml
import time

In [2]:
def get_access_token():
    """토큰 발급"""
    headers = {"content-type":"application/json"}
    body = {"grant_type":"client_credentials",
    "appkey":APP_KEY, 
    "appsecret":APP_SECRET}
    PATH = "oauth2/tokenP"
    URL = f"{URL_BASE}/{PATH}"
    res = requests.post(URL, headers=headers, data=json.dumps(body))
    ACCESS_TOKEN = res.json()["access_token"]
    return ACCESS_TOKEN

config_path_gcs = '/workspaces/codespaces-blank/project/stock/config.yaml'

with open(config_path_gcs, encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']

ACCESS_TOKEN = get_access_token()



In [3]:
def get_current_price(code: str="005930"):
    """현재가 조회"""
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    return int(res.json()['output']['stck_prpr'])

In [4]:
def get_support_line(code: str="005930", num_of_bar: int=15, term_of_bar: int=3):
    bar = list()    # 봉
    low = 0    # 각 봉의 최저
    support = 0    # 지지선

    end = time.time() + term_of_bar*60

    for b in range(num_of_bar):    # 봉의 개수 만큼 반복
        while time.time() <= end:    # 봉이 생성되는 시간 동안 현재가를 받아 가격봉 생성
            bar.append(get_current_price(code))
        bar.sort()    # 오름차순 정렬
        low += bar[0]    # 최저가
        bar = list()    # 봉 초기화

        end = time.time() + term_of_bar*60    # 다음 봉의 생성을 위해 시간 설정
    support = low/num_of_bar    # 지지선(최저의 평균)
    
    return support

In [ ]:
print(get_support_line("252670", 2, 1))

#### 저항선  
: 일정 시간 간격으로 생성되는 봉의 고점들의 평균

<br/>

> 저항선 구하기

In [5]:
def get_resistant_line(code: str="22005930", num_of_bar: int=15, term_of_bar: int=3):
    bar = list()    # 봉
    high = 0    # 각 봉의 최고
    resistance = 0    # 지지선

    end = time.time() + term_of_bar*60    # 반복할 시간

    for b in range(num_of_bar):    # 봉의 개수 만큼 반복
        while time.time() <= end:    # 봉이 생성되는 시간 동안 현재가를 받아 가격봉 생성
            bar.append(get_current_price(code))
        bar.sort(reverse=True)    # 오름차순 정렬
        high += bar[0]    # 최고가
        bar = list()    # 봉 초기화

        end = time.time() + term_of_bar*60    # 다음 봉의 생성을 위해 시간 설정
    resistance = high/num_of_bar    # 저항선(최고의 평균)
    
    return resistance

In [41]:
print(get_resistant_line("252670", 2, 1))

2745.0


#### 거래량 확인

<br/>  

> 실시간 거래량 구하기

In [6]:
import FinanceDataReader as fdr
import pandas as pd

In [15]:
test = fdr.DataReader("252670", "2023-10-01", "2023-11-07")
test.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-10-04,2810,2880,2810,2870,112919783,0.053211
2023-10-05,2830,2880,2830,2870,110344954,0.000000
2023-10-06,2860,2880,2835,2865,97731027,-0.001742
2023-10-10,2790,2860,2755,2850,115052992,-0.005236
2023-10-11,2765,2770,2710,2755,137405853,-0.033333


finance-datareader는 일간 데이터만 제공하므로 적합하지 않음  


한국투자증권 api를 활용해야 함

In [38]:
def get_acml_volume(code: str="005930"):    # 일간 누적 거래량
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)

    return int(res.json()['output']['acml_vol'])

In [18]:
print(get_acml_volume("252670"))

141879880


3분 간격으로 volume을 체크해야 함 

In [39]:
def get_mmty_volume(code: str="005930", term_of_bar: int=3):
    mmty_volume = 0
    base = get_acml_volume(code)

    end = time.time() + term_of_bar*60

    while time.time() <= end:
        acml = get_acml_volume(code)
        mmty_volume = acml-base
    
    return mmty_volume

In [48]:
print(get_mmty_volume("252670", 1))

0


문제: 지지선과 저항선, 거래량을 따로따로 구하면 세 가지 자료의 시점에 오차가 발생할 확률이 매우 높음  
&rarr; 세 가지 자료를 한 번에, 한 함수 안에서 구해야 함  
&rarr; 세 가지 데이터 및 저항선, 지지선을 구하는 프로그램을 따로 구현하여 계속 데이터를 업데이트 하도록 하기 

<br/>  

> 분봉, 저점, 고점, 분봉 생성 시의 거래량, 지지선, 저항선 구하기

In [1]:
import requests
import json
import yaml
import datetime
import time
from multiprocessing import Process

In [2]:
config_path_gcs = '/workspaces/codespaces-blank/project/stock/config.yaml'

with open(config_path_gcs, encoding='UTF-8') as f:
    _cfg = yaml.load(f, Loader=yaml.FullLoader)
APP_KEY = _cfg['APP_KEY']
APP_SECRET = _cfg['APP_SECRET']
ACCESS_TOKEN = ""
CANO = _cfg['CANO']
ACNT_PRDT_CD = _cfg['ACNT_PRDT_CD']
DISCORD_WEBHOOK_URL = _cfg['DISCORD_WEBHOOK_URL']
URL_BASE = _cfg['URL_BASE']
f = open("/workspaces/codespaces-blank/project/stock/national/yj/data/access_token.txt", "r")
ACCESS_TOKEN = f.readline()
f.close()

In [3]:
def send_message(msg):    # 디스코드 메시지 전송
    now = datetime.datetime.now()
    message = {"content": f"[{now.strftime('%Y-%m-%d %H:%M:%S')}] {str(msg)}"}
    requests.post(DISCORD_WEBHOOK_URL, data=message)
    print(message)

def get_acml_volume(code: str="005930"):    # 일간 누적 거래량
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)

    return int(res.json()['output']['acml_vol'])

def get_current_price(code: str="005930"):    # 현재가 조회
    PATH = "uapi/domestic-stock/v1/quotations/inquire-price"
    URL = f"{URL_BASE}/{PATH}"
    headers = {"Content-Type":"application/json", 
            "authorization": f"Bearer {ACCESS_TOKEN}",
            "appKey":APP_KEY,
            "appSecret":APP_SECRET,
            "tr_id":"FHKST01010100"}
    params = {
    "fid_cond_mrkt_div_code":"J",
    "fid_input_iscd":code,
    }
    res = requests.get(URL, headers=headers, params=params)
    
    return int(res.json()['output']['stck_prpr'])

In [4]:
class Stock_Info:
    def __init__(self, code: str="005930", term_of_bar: int=3, term_of_line: int=5, iteration: int=4, file_name: str="0"):
        self.code: str = code    # 종목 코드
        self.term_of_bar: int = term_of_bar    # 봉 생성 시간(분)
        self.term_of_line: int = term_of_line    # 지지선 및 저항선을 구할 때 필요한 봉의 개수
        self.iteration: int = iteration    # 반복 횟수
        self.file_name: str = file_name    # 파일 이름
        self.pr_low = list()    # 저점 리스트
        self.pr_high = list()    # 고점 리스트
        self.volume = 0    # 거래량
        self.support = 0    # 지지선
        self.resistant = 0    # 저항선

    def get_information(self):    # 종목의 저점, 고점, 거래량, 지지선, 저항선 구하기
        while self.support == 0:    # 지지선 및 저항선이 구해질 때까지 반복
            bar = list()    # 가격봉
            mmty_volume = 0    # 봉이 생성되는 동안의 거래량
            base = get_acml_volume(self.code)    # 현재까지의 누적 거래량(기준)
            end = time.time() + self.term_of_bar*5

            while time.time() <= end:    # 봉이 생성되는 시간 동안 현재가를 받아 가격봉 생성
                bar.append(get_current_price(self.code))
                acml = get_acml_volume(self.code)    # 현재까지의 누적 거래량(실시간)
                mmty_volume = acml-base    # 봉이 생성되는 동안의 거래량

            bar.sort()    # 오름차순 정렬
            self.pr_low.append(bar[0])    # 최저가를 리스트에 추가
            self.pr_high.append(bar[-1])    # 최고가를 리스트에 추가
            self.volume = mmty_volume    # 거래량

            f_volume = open("/workspaces/codespaces-blank/project/stock/national/yj/volume/volume"+self.file_name+".txt", "a")    # 거래량을 저장할 파일 열기
            f_volume.write(str(self.volume)+"\n")    # 거래량 파일에 쓰기
            f_volume.close()    # 파일 닫기

            if len(self.pr_low) >= self.term_of_line:    # 지지선 및 저항선을 구하는 조건
                self.get_support_resistant()    # 지지선 및 저항선 구하기

    def get_support_resistant(self):    # 종목의 지지선 및 저항선 구하기
        self.support = 0    # 지지선 초기화
        self.resistant = 0    # 저항선 초기화

        for i in range(len(self.pr_low)):    # 저점 리스트의 길이만큼 반복
            self.support += int(self.pr_low[i])    # 데이터 중 저점들의 합
            self.resistant += int(self.pr_high[i])    # 데이터 중 고점들의 합
        self.support /= self.term_of_line    # 지지선
        self.resistant /= self.term_of_line    #저항선

        self.pr_low = list()    # 저점 리스트 초기화
        self.pr_high = list()    # 고점 리스트 초기화

        f_sup = open("/workspaces/codespaces-blank/project/stock/national/yj/line/support"+self.file_name+".txt", "a")    # 지지선 값을 저장할 파일 열기
        f_res = open("/workspaces/codespaces-blank/project/stock/national/yj/line/resistant"+self.file_name+".txt", "a")    # 저항선 값을 저장할 파일 열기
        f_sup.write(str(self.support)+"\n")    # 지지선 값 파일에 쓰기
        f_res.write(str(self.resistant)+"\n")    # 저항선 값 파일에 쓰기
        f_sup.close()    # 파일 닫기
        f_res.close()    # 파일 닫기

        send_message(self.code+" "+"support: "+str(self.support))    # 지지선 값 메시지로 보내기
        send_message(self.code+" "+"resistant: "+str(self.resistant))    # 저항선 값 메시지로 보내기


In [6]:
while True:
    f_req = open("/workspaces/codespaces-blank/project/stock/national/yj/data/request.txt", "r")    # 요청 사항 파일 열기
    req = f_req.readlines()    # 요청 사항 읽기
    f_req.close()    # 파일 닫기
    for i in range(len(req)):
            req[i] = req[i].strip()    # 요청 사항 내용 중 개행 문자 제거

    info = list()    # 종목 정보 리스트
    for i in range(0, len(req)-2, 2):
        info.append(Stock_Info(req[i], int(req[i+1]), int(req[-2]), 4, str(i//2)))    # 각 종목에 대한 정보 클래스 인스턴스 생성

    procs = []    # CPU 작업 리스트
    for i in range(0, len(req)-2, 2):    # 요청 사항 중 종목 코드에만 접근
        proc = Process(target=info[i//2].get_information)    # 데이터 구하는 작업 병렬 수행
        procs.append(proc)
        proc.start()
    for proc in procs:
        proc.join()

[9610]
[9610]
[10565]
[10565]
[9595]
[9595]
[9610, 9610]
[9610, 9610]
[10565, 10565][9595, 9595]
[10565, 10565]

[9595, 9595]
[9595, 9595, 9595]
[9595, 9595, 9595]
hello
[9610, 9610, 9610]
[9610, 9610, 9610]
hello
[10565, 10565, 10565]
[10565, 10565, 10565]
hello
{'content': '[2023-11-19 13:18:48] 441640 support: 9610.0'}{'content': '[2023-11-19 13:18:48] 441680 support: 9595.0'}

{'content': '[2023-11-19 13:18:49] 453870 support: 10565.0'}
{'content': '[2023-11-19 13:18:49] 441640 resistant: 9610.0'}
{'content': '[2023-11-19 13:18:49] 441680 resistant: 9595.0'}
{'content': '[2023-11-19 13:18:49] 453870 resistant: 10565.0'}
[10565]
[10565]
[9595]
[9595][9610]

[9610]
[10565, 10565]
[10565, 10565]
[9595, 9595][9610, 9610]

[9595, 9595][9610, 9610]

[10565, 10565, 10565]
[10565, 10565, 10565]
hello
[9610, 9610, 9610]
[9610, 9610, 9610]
hello
[9595, 9595, 9595]
[9595, 9595, 9595]
hello
{'content': '[2023-11-19 13:19:07] 453870 support: 10565.0'}
{'content': '[2023-11-19 13:19:07] 441640 s

KeyboardInterrupt: 

[10565, 10565, 10565]
[10565, 10565, 10565]
hello
[9610, 9610, 9610]
[9610, 9610, 9610]
hello
[9595, 9595, 9595]
[9595, 9595, 9595]
hello
{'content': '[2023-11-19 13:20:18] 441640 support: 9610.0'}
{'content': '[2023-11-19 13:20:18] 453870 support: 10565.0'}
{'content': '[2023-11-19 13:20:18] 441680 support: 9595.0'}
{'content': '[2023-11-19 13:20:19] 441640 resistant: 9610.0'}
{'content': '[2023-11-19 13:20:19] 453870 resistant: 10565.0'}
{'content': '[2023-11-19 13:20:19] 441680 resistant: 9595.0'}


In [15]:
for i in range(len(procs)):
    print(procs[i])

<Process name='Process-19' pid=35033 parent=10689 stopped exitcode=0>
<Process name='Process-20' pid=35034 parent=10689 stopped exitcode=0>
<Process name='Process-21' pid=35035 parent=10689 stopped exitcode=0>


In [19]:
t_now = datetime.datetime.now()
print(type(t_now.minute))

<class 'int'>
